In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None
# plt.rcParams.update(
#     {
#         "lines.linewidth": 0.8,
#         "figure.figsize": [5,9],
#         "figure.dpi": 200,
#         "hatch.color": "grey",
#         "font.size": 6
#     }
# )


In [ ]:
from psy_simple.plotters import CMap, Bounds
from psy_maps.plotters import Transform, MapPlot2D, LonLatBox, MapPlotter
from psyplot.plotter import Plotter
import cartopy.crs as ccrs
import psyplot.project as psy
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cfgrib
import zarr
import glob
from PIL import Image
import iconarray
import os
from pyprojroot import here


In [ ]:
output_path = os.path.join(str(here()), 'output/gif/')

In [ ]:
ds = xr.open_zarr("/scratch/sadamov/pyprojects_data/aldernet/no_threshold/data_valid.zarr")
# ds = xr.open_dataset(str(here()) + "/data/pollen_ml.nc")

In [ ]:
with open(str(here()) + "/data/scaling.txt", "r", encoding="utf-8") as f:
    lines = [line.rstrip() for line in f]
    center = float(lines[0].split(": ")[1])
    scale = float(lines[1].split(": ")[1])

ds["ALNU"].values = np.maximum(0, ds["ALNU"].values * scale + center)

In [ ]:
plot1 = ds.psy.plot.mapplot(
    name="ALNU",
    valid_time=1100,
    sort=["valid_time"],
    title="Alder Pollen in the Alps in March",
    titlesize=15,
    lakes=True, 
    borders=True, 
    rivers=True, 
    grid_color="white",
    cticksize=8,
    grid_labelsize=8,
    projection = 'robin',
    cmap='RdBu_r')

colorbar = [x for x in range(0, int(ds["ALNU"].values.max()) + 200, 200)]
plot1.update(bounds=colorbar,
            cticks=colorbar)

plot1.show()


In [ ]:
for valid_time in range(len(ds.ALNU.valid_time)):
    plot1 = ds.psy.plot.mapplot(
        name="ALNU",
        valid_time=valid_time,
        sort=["valid_time"],
        title="Alder Pollen in the Alps",
        titlesize=15,
        lakes=True,
        borders=True,
        rivers=True,
        grid_color="white",
        cticksize=8,
        grid_labelsize=8,
        projection='robin',
        cmap='RdBu_r')

    colorbar = [x for x in range(0, int(ds["ALNU"].values.max()) + 200, 200)]
    plot1.update(bounds=colorbar,
                 cticks=colorbar)

    plt.ioff()
    plot1.export(output_path + '/map' + str(valid_time).zfill(4) + '.png')
    plt.ion()


In [ ]:
frames = [Image.open(image) for image in sorted(glob.glob(f"{output_path}/*.png"))]
frame_one = frames[0]
frame_one.save(output_path + "/my_awesome.gif", format="GIF", append_images=frames,
               save_all=True, duration=200, loop=0)
